# Librerias

In [1]:
# Librerias para el procesamiento de lenguaje natural

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import nltk

# Librerias para el procesamiento de archivos xml
import xml.etree.ElementTree as ET

# Librerias de uso general
from collections import defaultdict
import os
import re

# Librerias para tipado de funciones
from typing import List

# Configuración de palabras de parada
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Configuración de stemmer
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /home/nico/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Proceso de preprocesamiento

In [4]:
def preprocess(text: str) -> List[str]:
    # Normalización del texto
    text = text.lower()
    # Tokenización simple a nivel de palabra
    tokens = re.findall(r"\w+", text)
    # Quitar stopwords y aplicar stemming
    processed = [stemmer.stem(token) for token in tokens if token not in stop_words]
    # Retornar texto preprocesado
    return processed

In [ ]:
print(preprocess("The cats are running quickly in the gardens"))

['cat', 'run', 'quick', 'garden']


# Ingesta de documentos

In [ ]:
# Ruta en la que se encuentran los documentos a procesar
path_docs = "../data/test"

def load_documents(path_docs: str):
	# Diccionario para almacenar los documentos por id
	docs = {}
	# Se revisan únicamente los archivos relevantes
	for file_name in os.listdir(path_docs):
		if not file_name.endswith(".naf"):
			continue
		# Construccion del arbol de atributos del documento
		tree = ET.parse(os.path.join(path_docs, file_name))
		root = tree.getroot()
		# Id recuperado desde el atributo pupblicId
		doc_id = root.find("nafHeader/public").attrib["publicId"]
		# Titulo desde el atributo title
		title = root.find("nafHeader/fileDesc").attrib.get("title", "")
		# Contenido desde <raw>
		raw_element = root.find("raw")
		content = raw_element.text if raw_element is not None else ""
		# concatenar titulo + contenido para preprocesar
		tokens = preprocess(title +  " " + content)
		docs[doc_id] = tokens
	# Retornar los documentos cargados
	return docs

In [17]:
print(load_documents(path_docs))

{'d001': ['william', 'beaumont', 'human', 'digest', 'william', 'beaumont', 'human', 'digest', 'william', 'beaumont', 'physiolog', 'digest', 'imag', 'sourc', 'novemb', '21', '1785', 'us', 'american', 'surgeon', 'william', 'beaumont', 'born', 'becam', 'best', 'known', 'father', 'gastric', 'physiolog', 'follow', 'research', 'human', 'digest', 'william', 'beaumont', 'born', 'lebanon', 'connecticut', 'becam', 'physician', 'serv', 'surgeon', 'mate', 'armi', 'war', '1812', 'open', 'privat', 'practic', 'plattsburgh', 'new', 'york', 'rejoin', 'armi', 'surgeon', '1819', 'beaumont', 'station', 'fort', 'mackinac', 'mackinac', 'island', 'michigan', 'earli', '1820s', 'exist', 'protect', 'interest', 'american', 'fur', 'compani', 'fort', 'becam', 'refug', 'wound', '19', 'year', 'old', 'french', 'canadian', 'fur', 'trader', 'name', 'alexi', 'st', 'martin', 'shotgun', 'went', 'accid', 'american', 'fur', 'compani', 'store', 'close', 'rang', 'june', '6th', '1822', 'st', 'martin', 'wound', 'quit', 'serious

# Construcción de índice invertido

In [ ]:
def build_inverted_index(docs):
    # construcción de la estructura de indice invertido
    inverted = defaultdict(set)
    # Iteración sobre los doumentos cargados
    for doc_id, tokens in docs.items():
        # Iteración sobre los tokens de los documentos
        for token in tokens:
            # Población del índice
            inverted[token].add(doc_id)
    # Se retorna el índice construido
    return inverted